In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()
# DATASET_PATH=os.getenv("DATASET_DIRECTORY")
DATASET_PATH="/data/RadSAR"
CONFIG_DIRECTORY = os.getenv("CONFIG_DIRECTORY")

sys.path.append("../")
from mmwave_radar_processing.config_managers.cfgManager import ConfigManager
from cpsl_datasets.cpsl_ds import CpslDS

/home/david/.cache/pypoetry/virtualenvs/mmwave-radar-processing-ERvCMkmn-py3.12/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
cfg_manager = ConfigManager()

cfg_path = os.path.join(CONFIG_DIRECTORY,"6843_hand_ods_30Hz.cfg")
# cfg_path = os.path.join(CONFIG_DIRECTORY,"1843_RadVel_10Hz_lr_SAR.cfg")
cfg_manager.load_cfg(cfg_path)
cfg_manager.compute_radar_perforance(profile_idx=0)

print(f"range res: {cfg_manager.range_res_m}")
print(f"range max: {cfg_manager.range_max_m}")
print(f"profile config: \n\t{cfg_manager.profile_cfgs[0]}")
print(f"Frame loops: {cfg_manager.frameCfg_loops}")
print(f"vel res: {cfg_manager.vel_res_m_s}")
print(f"vel max: {cfg_manager.vel_max_m_s}")
print(f"Chirp bandwidth: {cfg_manager.profile_cfgs[0]['freqSlope_MHz_us'] * cfg_manager.profile_cfgs[0]['rampEndTime_us']} MHz")
chirp_period_us = cfg_manager.profile_cfgs[0]['idleTime_us'] + cfg_manager.profile_cfgs[0]['rampEndTime_us']
print(f"Chirp period: {chirp_period_us} us")

num_tx_antennas = cfg_manager.frameCfg_end_index - cfg_manager.frameCfg_start_index + 1
active_frame_period = num_tx_antennas * cfg_manager.frameCfg_loops * chirp_period_us * 1e-3
print(f"Active frame time: {active_frame_period} ms")
print(f"Frame period: {cfg_manager.frameCfg_periodicity_ms}")


range res: 0.04745632415959596
range max: 3.321942691171717
profile config: 
	{'profileId': 0, 'startFreq_GHz': 60.0, 'idleTime_us': 120.0, 'adcStartTime_us': 7.0, 'rampEndTime_us': 40.0, 'txOutPower': 0.0, 'txPhaseShifter': 0.0, 'freqSlope_MHz_us': 99.0, 'txStartTime_us': 1.0, 'adcSamples': 70, 'sampleRate_kSps': 2194, 'hpfCornerFreq1': 0, 'hpfCornerFreq2': 0, 'rxGain_dB': 158.0}
Frame loops: 40
vel res: 0.1301182543402778
vel max: 2.602365086805556
Chirp bandwidth: 3960.0 MHz
Chirp period: 160.0 us
Active frame time: 19.2 ms
Frame period: 33.3


In [3]:
freq = 60e9
lbda = 3e8 / (freq)
T_chirp = 160e-6
N_chirps = 40
num_tx = 3

slope = 99e12
chirp_bw = 3.95e9
chirp_period = chirp_bw / slope
sampled_period = chirp_period - 8e-6
sampled_bw = slope * sampled_period
idle_time = T_chirp - chirp_period

num_samples = 70
range_res = 3e8 / (2 * sampled_bw)
sample_rate = num_samples / sampled_period
range_max = num_samples * range_res

#computing frame characteristics
frame_duration = N_chirps * T_chirp * num_tx

#computing velocity
v_max = lbda / (4 * T_chirp * num_tx)
v_res = lbda / (2 * N_chirps * T_chirp * num_tx)

print(f"Frame duration: {frame_duration*1e3} ms")
print(f"Wavelength: {lbda*1e3} mm")
print(f"Chirp duration: {T_chirp*1e6} us")
print(f"Max velocity: {v_max} m/s")
print(f"Velocity resolution: {v_res} m/s")
print(f"Range resolution: {range_res} m")
print(f"Max range: {range_max} m")
print(f"Sampled bandwidth: {sampled_bw/1e9} GHz")
print(f"Chirp period (from slope): {chirp_period*1e6} us")
print(f"Idle time: {idle_time*1e6} us")
print(f"Sample rate: {sample_rate/1e6} MHz")


Frame duration: 19.200000000000003 ms
Wavelength: 5.0 mm
Chirp duration: 160.0 us
Max velocity: 2.6041666666666665 m/s
Velocity resolution: 0.13020833333333331 m/s
Range resolution: 0.04749841671944269 m
Max range: 3.324889170360988 m
Sampled bandwidth: 3.158 GHz
Chirp period (from slope): 39.898989898989896 us
Idle time: 120.10101010101012 us
Sample rate: 2.194426852438252 MHz
